### 下载VGG

In [ ]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm

vgg_dir = 'tensorflow_vgg/'
# Make sure vgg exists
if not isdir(vgg_dir):
    raise Exception("VGG directory doesn't exist!")

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(vgg_dir + "vgg16.npy"):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc='VGG16 Parameters') as pbar:
        urlretrieve(
            'https://s3.amazonaws.com/content.udacity-data.com/nd101/vgg16.npy',
            vgg_dir + 'vgg16.npy',
            pbar.hook)
else:
    print("Parameter file already exists!")

In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow_vgg import vgg16
from tensorflow_vgg import utils
import glob
data_dir = 'flower_photos/'
classes = glob.glob(data_dir+ r'*')
classes_ = []
for class_ in classes:
    class_ = class_[14:]
    classes_.append(class_)

### 得到各种种类的花各有多少

In [ ]:
for each in classes_:
    print(each)
    file_paths = glob.glob(data_dir + each + r'/*')
    print(len(file_paths))

### 对所有的花朵旋转10度，1变十

In [ ]:
import cv2
for each in classes_:
    file_paths = glob.glob(data_dir + each + r'/*')
    for path in file_paths:
        img = cv2.imread(path)
        height,width=img.shape[:2]
        degree=0
            #旋转后的尺寸
        for degree in range(0,10):
            heightNew=int(width*fabs(sin(radians(degree)))+height*fabs(cos(radians(degree))))
            widthNew=int(height*fabs(sin(radians(degree)))+width*fabs(cos(radians(degree))))

            matRotation=cv2.getRotationMatrix2D((width/2,height/2),degree,1)

            matRotation[0,2] +=(widthNew-width)/2  #重点在这步，目前不懂为什么加这步
            matRotation[1,2] +=(heightNew-height)/2  #重点在这步

            imgRotation=cv2.warpAffine(img,matRotation,(widthNew,heightNew),borderValue=(255,255,255))
            cv.imwrite(path[:-4] + str(degree) + '.jpg' , imgRotation)

### 搭建迁移学习模型，进行预训练

In [ ]:

batch_size = 30
codes_list = []
labels = []
batch = []

codes = None

with tf.Session() as sess:
    vgg = vgg16.Vgg16()
    input_ = tf.placeholder(tf.float32, [None, 224, 224, 3])
    with tf.name_scope("content_vgg"):
        vgg.build(input_)

    for each in classes_:
        print("Starting {} images".format(each))
        class_path = data_dir + each
        files = os.listdir(class_path)
        for ii, file in enumerate(files, 1):
            # Add images to the current batch
            # utils.load_image crops the input images for us, from the center
            img = cv2.reshape(img,(224,224))
            batch.append(img.reshape((1, 224, 224, 3)))
            labels.append(each)
            
            # Running the batch through the network to get the codes
            if ii % batch_size == 0 or ii == len(files):
                images = np.concatenate(batch)

                feed_dict = {input_: images}
                codes_batch = sess.run(vgg.relu6, feed_dict=feed_dict)
                
                # Here I'm building an array of the codes
                if codes is None:
                    codes = codes_batch
                else:
                    codes = np.concatenate((codes, codes_batch))
                
                # Reset to start building the next batch
                batch = []
                print('{} images processed'.format(ii))

### 将处理好的纹理放入文件保存

In [ ]:
with open('codes', 'w') as f:
    codes.tofile(f)
    
# write labels to file
import csv
with open('labels', 'w') as f:
    writer = csv.writer(f, delimiter='\n')
    writer.writerow(labels)